In [38]:
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [39]:
from langchain_openai import OpenAIEmbeddings

In [40]:
from langchain_community.document_loaders import PyPDFLoader

In [41]:
from langchain_chroma import Chroma

In [42]:
from langchain.retrievers.multi_query import MultiQueryRetriever

In [43]:
from langchain_openai import ChatOpenAI

In [44]:
from langchain.prompts import PromptTemplate

In [45]:
from langchain.chains import LLMChain

In [46]:
#loader = PyPDFLoader("C:\\Users\\Admin\\Desktop\\GenAI\\MultiQuery\\data2\\data2.pdf")
loader = PyPDFLoader(".\\data2\\data2.pdf")

In [47]:
from langchain.chat_models import ChatOpenAI

In [48]:
pages=loader.load_and_split()

In [49]:
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=pages, embedding=embedding)

In [50]:
import os

In [51]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"])

C:\Users\Admin\AppData\Local\Temp\ipykernel_10032\1075850449.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"])


In [52]:
retriever=MultiQueryRetriever.from_llm(retriever=vectordb.as_retriever(),llm=llm)

In [53]:
import logging
logging.basicConfig()
logger=logging.getLogger("langchain.retrievers.multiquery")
logger.setLevel(logging.INFO)

In [54]:
question="what is radiation and what are Photovoltaic detectors"

docs=retriever.get_relevant_documents(query=question)

In [55]:
len(docs)

2

In [56]:
QA_PROMPT= PromptTemplate(
                         input_variables=["query","contexts"],
                         template="""You are a helpful assistant. You have to answer the provided query",
                         Contexts:{contexts},
                         Question:{query}"""
                         )

In [57]:
qa_chain = LLMChain(llm=llm, prompt=QA_PROMPT)

In [58]:
out = qa_chain(
                inputs={
                    "query":question,
                    "contexts" :"\n------\n".join(d.page_content for d in docs)
                }
)
out["text"]

'Radiation refers to the emission of energy in the form of electromagnetic waves or particles. In the context of solar radiation, it specifically refers to the energy emitted by the sun in the form of electromagnetic waves.\n\nPhotovoltaic detectors are devices that convert solar radiation into electrical energy. They have the additional characteristics of having an instantaneous and linear response to changing radiation levels. They also have a temperature dependence of approximately 0.15% per degree Celsius maximum. These detectors are typically fitted with an acrylic diffuser to reduce the dependence of response on the angle of incidence of the radiation.'